In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pyswarms as ps

In [2]:
X_train = np.load('X_train.npy')
X_test  = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test  = np.load('y_test.npy')

In [3]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Алгоритм роя частиц

## KNN

In [ ]:
def objective_knn(x):
    n_particles = x.shape[0]
    scores = np.zeros(n_particles)
    for i in range(n_particles):
        n_neighbors = int(np.round(x[i, 0]))
        n_neighbors = max(1, n_neighbors)
        p = int(np.round(x[i, 1]))
        p = 1 if p < 1 else 2 if p > 2 else p

        model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p, n_jobs=-1)
        score = cross_val_score(
            model,
            X_train,
            y_train,
            cv=cv,
            scoring='f1_macro',
            n_jobs=-1
        ).mean()
        scores[i] = -score  
    return scores

In [5]:
# Параметры PSO
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
# Границы: n_neighbors [1,50], p [1,10]
bounds_knn = (np.array([1, 1]), np.array([50, 10]))

In [6]:
optimizer_knn = ps.single.GlobalBestPSO(
    n_particles=20,
    dimensions=2,
    options=options,
    bounds=bounds_knn
)

In [ ]:
best_cost_knn, best_pos_knn = optimizer_knn.optimize(objective_knn, iters=50)

In [ ]:
best_n_neighbors = int(np.round(best_pos_knn[0]))
best_p = int(np.round(best_pos_knn[1]))
print("Best KNN params: n_neighbors=", best_n_neighbors, ", p=", best_p)
print("Best KNN f1_macro=", -best_cost_knn)

## SVC

In [ ]:
def objective_svc(x):
    n_particles = x.shape[0]
    scores = np.zeros(n_particles)
    for i in range(n_particles):
        C = 10 ** x[i, 0]
        gamma = 10 ** x[i, 1]

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svc', SVC(C=C, gamma=gamma, kernel='rbf', random_state=42))
        ])
        score = cross_val_score(
            pipeline,
            X_train,
            y_train,
            cv=cv,
            scoring='f1_macro',
            n_jobs=-1
        ).mean()
        scores[i] = -score
    return scores

In [9]:
# Границы: log10(C) [-3,3], log10(gamma) [-5,2]
bounds_svc = (np.array([-3, -5]), np.array([3, 2]))

optimizer_svc = ps.single.GlobalBestPSO(
    n_particles=20,
    dimensions=2,
    options=options,
    bounds=bounds_svc
)

In [ ]:
best_cost_svc, best_pos_svc = optimizer_svc.optimize(objective_svc, iters=50)

In [ ]:
best_C = 10 ** best_pos_svc[0]
best_gamma = 10 ** best_pos_svc[1]
print("Best SVC params: C=", best_C, ", gamma=", best_gamma)
print("Best SVC f1_macro=", -best_cost_svc)